In [6]:
from collections import defaultdict
from pathlib import Path
import json

import pandas as pd

In [4]:
data_dir = Path.cwd().parent / 'data' / 'working' / '2018-05-01' / 'idn-tagged'

In [7]:
overall_f1_scores = defaultdict(list)
for model_name in 'majority memo crf'.split():
    for fold in range(1, 6):
        result_path = data_dir / f'result-{model_name}.{fold:02}.json'
        with open(result_path) as f:
            result = json.load(f)
        overall_f1_scores[model_name].append(result['overall_f1'])

In [10]:
df = pd.DataFrame(overall_f1_scores, index=range(1, 6))

In [11]:
df

,crf,majority,memo
1,0.960914,0.092034,0.920352
2,0.964923,0.092209,0.899710
3,0.962995,0.093065,0.904711
4,0.950597,0.095158,0.904421
5,0.962167,0.096895,0.901569


In [12]:
df.describe()

,crf,majority,memo
count,5.000000,5.000000,5.000000
mean,0.960319,0.093872,0.906152
std,0.005627,0.002097,0.008204
min,0.950597,0.092034,0.899710
25%,0.960914,0.092209,0.901569
50%,0.962167,0.093065,0.904421
75%,0.962995,0.095158,0.904711
max,0.964923,0.096895,0.920352
